The best dataset I've found so far contains is tatoeba.

https://tatoeba.org/fr/downloads

Another is conversational sentences from film and series subtitles with translations for multiple languages:

https://opus.nlpl.eu/OpenSubtitles-v2018.php
https://github.com/PolyAI-LDN/conversational-datasets

In [2]:
import os
import constants
import pandas as pd

In [3]:
constants.language_code = 'de'

In [4]:
#filepath_en = f"../input_files/{constants.language_code}/open_subtitles/OpenSubtitles_en-{constants.language_code}.en"
#filepath_lang = f"../input_files/{constants.language_code}/open_subtitles/OpenSubtitles_en-{constants.language_code}.{constants.language_code}"

filepath = f"../input_files/{constants.language_code}/tatoeba_{constants.language_code}_en.tsv"

df = pd.read_csv(filepath, sep='\\t')
#en_series = pd.read_csv(filepath_en, sep='\\t')
#lang_series = pd.read_csv(filepath_lang, sep='\\t')

C:\Users\Toby Usher\AppData\Local\Temp\ipykernel_14444\63767765.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filepath, sep='\\t')


In [5]:
df.head()

,243,Один раз в жизни я делаю хорошее дело... И оно бесполезно.,3257,For once in my life I'm doing a good deed... And it is useless.
0,5409,Давайте что-нибудь попробуем!,1276,Let's try something.
1,5410,Мне пора идти спать.,1277,I have to go to sleep.
2,5411,Что ты делаешь?,16492,What are you doing?
3,5411,Что ты делаешь?,511884,What do you make?
4,5411,Что ты делаешь?,1839618,What're you doing?


In [6]:
df.count

<bound method DataFrame.count of              243 Один раз в жизни я делаю хорошее дело... И оно бесполезно.  \
0           5409                      Давайте что-нибудь попробуем!           
1           5410                               Мне пора идти спать.           
2           5411                                    Что ты делаешь?           
3           5411                                    Что ты делаешь?           
4           5411                                    Что ты делаешь?           
...          ...                                                ...           
716082   2134530                      Интересно, какой путь короче.           
716083  12434553                         Ноэль празднует Рождество.           
716084  12434552                          Ноэль отмечает Рождество.           
716085  12434561                         Смотрите, что Том мне дал.           
716086  12434560                           Смотри, что Том мне дал.           

            3257  

In [7]:
# Only keep the sentence and translation columns
df = pd.DataFrame({
    'sentence': df.iloc[:, 1],
    'translated_sentence': df.iloc[:, 3]
})

In [8]:
df.head()

,sentence,translated_sentence
0,Давайте что-нибудь попробуем!,Let's try something.
1,Мне пора идти спать.,I have to go to sleep.
2,Что ты делаешь?,What are you doing?
3,Что ты делаешь?,What do you make?
4,Что ты делаешь?,What're you doing?


In [9]:
"""# Combine into a dataframe and randomly sample n rows
n = 3_000_000

lang_series = lang_series.reset_index(drop=True)
en_series = en_series.reset_index(drop=True)

lang_series_list = lang_series.values.tolist()
en_series_list = en_series.values.tolist()

# Flatten the lists
lang_series_list = [item[0] for item in lang_series_list]
en_series_list = [item[0] for item in en_series_list]


df = pd.DataFrame({
    'sentence': lang_series_list,
    'translation': en_series_list
})

# First 100 or so lines are in English for some reason
#df = df[df.index > 100]

df_sample = df.sample(n, random_state=1)
df_sample.to_csv(f'../input_files/{constants.language_code}/open_subtitles_uncleaned_sentences.csv', sep='\t')
"""

"# Combine into a dataframe and randomly sample n rows\nn = 3_000_000\n\nlang_series = lang_series.reset_index(drop=True)\nen_series = en_series.reset_index(drop=True)\n\nlang_series_list = lang_series.values.tolist()\nen_series_list = en_series.values.tolist()\n\n# Flatten the lists\nlang_series_list = [item[0] for item in lang_series_list]\nen_series_list = [item[0] for item in en_series_list]\n\n\ndf = pd.DataFrame({\n    'sentence': lang_series_list,\n    'translation': en_series_list\n})\n\n# First 100 or so lines are in English for some reason\n#df = df[df.index > 100]\n\ndf_sample = df.sample(n, random_state=1)\ndf_sample.to_csv(f'../input_files/{constants.language_code}/open_subtitles_uncleaned_sentences.csv', sep='\t')\n"

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 716087 entries, 0 to 716086
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   sentence             716087 non-null  object
 1   translated_sentence  716087 non-null  object
dtypes: object(2)
memory usage: 10.9+ MB


In [11]:
# Let's see if there are any duplicates in the dataset
df[df["sentence"].duplicated(keep=False)].sort_values("sentence").count

<bound method DataFrame.count of                                                  sentence  \
148751                  "А где начальство?" - "На обеде".   
151739                  "А где начальство?" - "На обеде".   
610451                          "Вернись!" - закричал он.   
14757                           "Вернись!" - закричал он.   
29971   "Возьми единственного сына твоего, которого ты...   
...                                                   ...   
332514                                         Ящик пуст.   
179240                                       Ящик пустой.   
332515                                       Ящик пустой.   
19039   —Что происходит в пещере? Мне интересно. —Поня...   
19040   —Что происходит в пещере? Мне интересно. —Поня...   

                                      translated_sentence  
148751         "And where are the bosses?" – "At dinner".  
151739            "And where is the boss?" - "At dinner".  
610451                             "Come back!" he cri

In [12]:
df.dtypes

sentence               object
translated_sentence    object
dtype: object

In [13]:
# Remove all duplicates from the dataframe
df = df.drop_duplicates("sentence")

In [14]:
df.info

<bound method DataFrame.info of                                             sentence  \
0                      Давайте что-нибудь попробуем!   
1                               Мне пора идти спать.   
2                                    Что ты делаешь?   
5                                           Что это?   
9       Сегодня 18 июня, и это день рождения Мюриэл!   
...                                              ...   
716082                 Интересно, какой путь короче.   
716083                    Ноэль празднует Рождество.   
716084                     Ноэль отмечает Рождество.   
716085                    Смотрите, что Том мне дал.   
716086                      Смотри, что Том мне дал.   

                                     translated_sentence  
0                                   Let's try something.  
1                                 I have to go to sleep.  
2                                    What are you doing?  
5                                            What is it?  


In [15]:
lengths: pd.Series = df['sentence'].str.len()
max_characters: int = lengths.max()
max_index = lengths.idxmax()

# Find the sentence with the most characters to see if there are any delimitation issues.
print(f'Longest sentence: {max_characters} characters')
print(df[df.index == max_index]['sentence'].values[0][:600]) # Print out first 600 characters

Longest sentence: 837 characters
Я еврей. Да разве у еврея нет глаз? Разве у еврея нет рук, внутренних органов, частей тела, чувств, привязанностей, страстей? Разве не та же самая пища питает его, не то же оружие ранит его, не те же болезни поражают его, не же средства лечат его, не так же знобит зима, не так же греет лето, что и христианина? Когда нас колют, разве из нас не течёт кровь? Когда нас щекочут, разве мы не смеёмся? Когда нас отравляют, разве мы не умираем? А когда нас оскорбляют, разве мы не должны мстить? Если мы во всё похожи на вас, то мы хотим походить и в этом. Если еврей оскорбит христианина, что внушает том


In [16]:
# Cut off any sentences longer than 200 or 
# shorter than 30 characters
df = df[
    (df['sentence'].str.len() < 200)
    & (df['sentence'].str.len() > (15 if constants.language_code == 'th' else 30)) # setting to 30 cuts out most thai sentences
     ]


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 270832 entries, 9 to 716081
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   sentence             270832 non-null  object
 1   translated_sentence  270832 non-null  object
dtypes: object(2)
memory usage: 6.2+ MB


In [18]:
# Randomly sample n rows to get a reduced dataset for easier training while testing out this method. Set a seed for reproducability.
#n_rows = 30000

#reduced_df = df.sample(n=n_rows, random_state=1)

In [19]:
# Remove id column and save dataframes as csv
df.to_csv(f"../output_files/{constants.language_code}/step0_sentences.csv", sep='\t', index=False)
#reduced_df.to_csv("./french_sentences_reduced.csv", sep='\t', index=False)